In [ ]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

# CAD Viewer

## Auto show in cells

In [ ]:
box = cq.Workplane('XY').box(1, 2, 3).edges().fillet(0.1)
box

## Use `show`

In [ ]:
cv = open_viewer("CadQuery", anchor="right")  # sets default viewer

In [ ]:
box

In [ ]:
show(box, viewer=None)

In [ ]:
cv2 = open_viewer("CadQuery2", anchor="split-bottom") 

In [ ]:
cv.close()

In [ ]:
close_viewers()

In [ ]:
show?

In [ ]:
cv = show(
    box,
    
    viewer="CadQuery",
    anchor="right",
    cad_width=640,
    tree_width=250,
    height=480,
    theme="light",
    pinning=False,
    
    angular_tolerance=0.1,
    deviation=1,
    edge_accuracy=0.01,
    default_color=(23, 176, 36),
    default_edge_color="#FF0000",
    optimal_bb=True,
    render_normals=True,
    render_edges=True,
    render_mates=True,
    mate_scale=2,

    control='trackball',
    axes=True,
    axes0=True,
    grid=[True, True,False],
    ticks=5,
    ortho=False,
    transparent=True,
    black_edges=False,
    position=(12.4, 7.7, -6.9),
    quaternion=(0.522, 0.611, 0.582, 0.120),
    target=(0.4, -1.6, -2.2),
    zoom=0.66,
    reset_camera=True,
    zoom_speed=3,
    pan_speed=3,
    rotate_speed=3,
    ambient_intensity=0.5,
    direct_intensity=0.05,
    show_parent=True,
    tools=True,
    timeit=False,
    js_debug=False
    
)

# Assemblies

In [ ]:
cv = open_viewer("Assembly", cad_width=720, height=540)
set_defaults(viewer="Assembly")

In [ ]:
box1 = cq.Workplane('XY').box(10, 20, 30).edges(">X or <X").chamfer(2)
box1.name = "box1"

box2 = cq.Workplane('XY').box(8, 18, 28).edges(">X or <X").chamfer(2)
box2.name = "box2"

box3 = cq.Workplane('XY').transformed(offset=(0, 15, 7))\
    .box(30, 20, 6).edges(">Z").fillet(3)
box3.name = "box3"

box4 = box3.mirror("XY").translate((0, -5, 0))
box4.name = "box4"

box1 = box1\
    .cut(box2)\
    .cut(box3)\
    .cut(box4)

In [ ]:
col = lambda w: cq.Color(*[int(w[i:i+2], 16) / 255 for i in (1,3,5)])

a1 = (
    cq.Assembly(name="ensemble")
    .add(box1, name="red box", color=col("#d7191c"))
    .add(box3, name="gree box", color=col("#abdda4"))
    .add(box4, name="blue box", color=col("#2b83ba"))
)

cv = show(a1, axes=True, grid=[True, False, False], ortho=True, axes0=True, collapse=1)

In [ ]:
show(a1, glass=True, transparent=True)

# Faces, Edges, Vertices

In [ ]:
box1

In [ ]:
box1.faces("not(|Z or |X or |Y)")

In [ ]:
box1.edges("not(|X or |Y or |Z)")

In [ ]:
box1.vertices()

In [ ]:
show(
    box1, 
    box1.faces("not(|Z or |X or |Y)"), 
    box1.edges("not(|X or |Y or |Z)"), 
    box1.vertices(),
    show_parent=False
)

# Replay

In [ ]:
cv = open_viewer("Replay")

In [ ]:
r = replay(box1)

In [ ]:
disable_replay()

# Camera position control

In [ ]:
cv = open_viewer("Assembly", cad_width=720)

show(
    a1, 
    control="orbit",
)

In [ ]:
p0 = cv.position
t0 = cv.target
q0 = cv.quaternion
z0 = cv.zoom
p0, q0, t0, z0

In [ ]:
cv.position = (135.3, 79.2, 29.6)
cv.target = (-4.8, 17.1, 7.9)
cv.quaternion = (0.399, 0.518, 0.643, 0.396)
cv.zoom = 1.45

In [ ]:
cv.position = p0

if cv.control == "trackball":
    cv.quaternion = q0

cv.target = t0
cv.zoom = z0

# Widget interaction

In [ ]:
cv.update_states({
    '/ensemble/red box/red box_0': (1,0),
    '/ensemble/blue box/blue box_0': [0,1],
})

In [ ]:
cv.update_states({
    '/ensemble/red box/red box_0': (1,1),
    '/ensemble/blue box/blue box_0': [1,1],
})

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.default_edge_color
cv.default_edge_color = "#ff0000"

In [ ]:
cv.default_edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.grid = [not g for g in cv.widget.grid]
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges
cv.tools = not cv.tools
cv.ortho = not cv.ortho

In [ ]:
cv.glass = not cv.glass

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed = 1
cv.pan_speed = 1
cv.rotate_speed = 1

In [ ]:
cv.last_pick

In [ ]:
pick = get_pick(a1, cv.last_pick)

if pick is not None:
    show(pick, viewer=None)
else:
    print("Double click on element in the CAD view first")

# Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = True

In [ ]:
cv.clip_planes = True

In [ ]:
cv.clip_value_0 = 0
cv.clip_value_1 = -2
cv.clip_value_2 = 12

In [ ]:
cv.clip_intersection = False

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.clip_planes = False

In [ ]:
cv.select_tree()

# Rotations

## Trackball

In [ ]:
cv = show(a1, viewer="Cadquery", control="trackball")

In [ ]:
import time

for i in range(10):
    cv.rotate_x(1)
    cv.rotate_y(3)
    cv.rotate_z(5)
    time.sleep(0.1)

In [ ]:
for i in range(10):
    cv.rotate_z(-5)
    cv.rotate_y(-3)
    cv.rotate_x(-1)
    time.sleep(0.1)    

## Orbit

In [ ]:
cv = show(a1, viewer="Cadquery", control="orbit")

In [ ]:
for i in range(10):
    cv.rotate_up(3)
    cv.rotate_left(1)
    time.sleep(0.1)

In [ ]:
for i in range(10):
    cv.rotate_up(-3)
    cv.rotate_left(-1)
    time.sleep(0.1)    

# Exports

## PNG

In [ ]:
cv.export_png("boxes.png")


## HTML

**Note**: This will create a file `cadquery.html` in the same folder as the notebook.
Doubleclicking will not work in Chrome (CORB issue), however it works in Firefox and Safari (at least on a Mac)



In [ ]:
cv = show(a1, viewer=None, control="orbit")

In [ ]:
cv.export_html()

## STL

**Note**: This only works for `PartGroup`s. For an<y CadQuery native object, use CadQuery's methods

In [ ]:
import cadquery as cq
from jupyter_cadquery.export import exportSTL
from jupyter_cadquery import Part, PartGroup, show

box = cq.Workplane().box(10,20,30).edges().fillet(4)
sphere = cq.Workplane().sphere(12).translate((0,0,10))
pg = PartGroup([Part(box),Part(sphere)])

exportSTL(
    pg,
    "box.stl", 
    tolerance=0.01, 
    angular_tolerance=0.2
)

# three-cad-viewer Format

In [ ]:
from jupyter_cadquery.cad_objects import exportJson
exportJson(a1, "boxes.json")

In [ ]:
close_viewers()